In [1]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt

# !pip3 install openpyxl

## Helper Functions


In [2]:
def extract_excel_sheet(excel_files, sheet_name, output_folder, csv_filename):
    """
    Extract a specific sheet from multiple Excel files, combine them into one DataFrame, and save as CSV.

    Parameters:
    - excel_files (list): List of Excel file paths.
    - sheet_name (str): Name of the sheet to extract (must exist in the file).
    - output_folder (str): Folder where CSV will be saved.
    - csv_filename (str): Name of the output CSV file.

    Returns:
    - pd.DataFrame: The combined DataFrame.
    """
    os.makedirs(output_folder, exist_ok=True)
    all_dfs = []

    for file in excel_files:
        xls = pd.ExcelFile(file)
        if sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            df['Source_File'] = os.path.basename(file)
            all_dfs.append(df)
        else:
            print(f"Sheet '{sheet_name}' not found in {file}")

    if not all_dfs:
        print(" No data extracted.")
        return pd.DataFrame()

    combined_df = pd.concat(all_dfs, ignore_index=True)
    csv_path = os.path.join(output_folder, csv_filename)
    combined_df.to_csv(csv_path, index=False)
    print(f"Saved {len(combined_df)} rows from sheet '{sheet_name}' to {csv_path}")
    return combined_df

In [3]:
excel_files = [
    '../data/raw/2016_12_31.xlsx',
    '../data/raw/2017_01_2021_03.xlsx',
    '../data/raw/2021_03_2023_12.xlsx'
]
output_folder = '../data/csv'


### Concacanate Excel files for output and author


In [4]:
# Step 1: Load df_output
df_output = extract_excel_sheet(
    excel_files=excel_files,
    sheet_name='ResearchOutput',
    output_folder=output_folder,
    csv_filename='df_output_combined.csv'
)

# Step 2: Load df_author
df_author = extract_excel_sheet(
    excel_files=excel_files,
    sheet_name='Addresses and Names',
    output_folder=output_folder,
    csv_filename='df_author_combined.csv'
)

Saved 28283 rows from sheet 'ResearchOutput' to ../data/csv/df_output_combined.csv
Saved 194724 rows from sheet 'Addresses and Names' to ../data/csv/df_author_combined.csv


In [5]:
df_output = pd.read_csv("../data/csv/df_output_combined.csv")
df_author = pd.read_csv("../data/csv/df_author_combined.csv")

columns_to_keep = {
    'Accession Number (UT)': 'paperId',
    '1st Document Type': 'docType',
    'Keywords': 'keywords',
    'Keywords Plus': 'keywordsPlus',
    '1st Subject Sub-Heading': 'subjectSub1',
    '1st Subject Category (traditional)': 'subjectCat1',
    'Subject Category (extended)': 'subjectCatExt',
    '1st Category Heading': 'categoryHeading',
    'Abstract': 'abstract',
    'Published Year': 'year',
    'DOI': 'doi',
    'Source title': 'sourceTitle',
    'Title': 'title'
}

columns_to_keep_authors = {
    'Accession Number (UT)': 'paperId',
    'Country': 'country',
    'Location': 'location',
    'Organisation names (concatenated)': 'orgNames',
    '1st Enhanced Organisation name': 'orgEnhanced1',
    'Enhanced Organisation Names (concatenated)': 'orgEnhancedAll',
    'Full Name': 'fullName',
    'ResearcherID': 'researcherId',
    'Researcher/Author SeqNo (position)': 'authorSeq'
}

existing_cols = [col for col in columns_to_keep if col in df_output.columns]
df_output = df_output[existing_cols].rename(columns={col: columns_to_keep[col] for col in existing_cols})

existing_cols_authors = [col for col in columns_to_keep_authors if col in df_author.columns]
df_author = df_author[existing_cols_authors].rename(
    columns={col: columns_to_keep_authors[col] for col in existing_cols_authors})

print("df_output columns:", df_output.columns.tolist())
print("df_author columns:", df_author.columns.tolist())

df_output columns: ['paperId', 'docType', 'keywords', 'keywordsPlus', 'subjectSub1', 'subjectCat1', 'subjectCatExt', 'categoryHeading', 'abstract', 'year', 'doi', 'sourceTitle', 'title']
df_author columns: ['paperId', 'country', 'location', 'orgNames', 'orgEnhanced1', 'orgEnhancedAll', 'fullName', 'researcherId', 'authorSeq']


In [6]:
# Step 1: Filter df_output by publication year (2007–2023)
before_filter_output = len(df_output)
df_output['year'] = pd.to_numeric(df_output['year'], errors='coerce')  # Ensure year is numeric
df_output_cleaned = df_output[df_output['year'].between(2007, 2023, inclusive='both')]  # Keep only valid years
after_filter_output = len(df_output_cleaned)
print(f"Filtered df_output: {before_filter_output} → {after_filter_output} (2007–2023)")

# Step 2: Filter df_author to keep only entries whose paperId exists in the cleaned df_output
valid_paper_ids = set(df_output_cleaned['paperId'].dropna().unique())  # Collect valid paperIds
before_filter_author = len(df_author)
df_author_cleaned = df_author[df_author['paperId'].isin(valid_paper_ids)]  # Filter author data accordingly
after_filter_author = len(df_author_cleaned)
print(f"Filtered df_author: {before_filter_author} → {after_filter_author} (matching paperIds from df_output_cleaned)")

# Step 3: Identify and report paperIds that are in df_output_cleaned but have no matching entries in df_author
paper_ids_output = set(df_output_cleaned['paperId'].dropna().unique())
paper_ids_author = set(df_author_cleaned['paperId'].dropna().unique())
missing_in_author = paper_ids_output - paper_ids_author  # Difference = missing links

print(f"\nUnique paperIds in df_output_cleaned: {len(paper_ids_output)}")
print(f"Unique paperIds in df_author_cleaned: {len(paper_ids_author)}")
print(f"PaperIds in df_output_cleaned but missing in df_author_cleaned: {len(missing_in_author)}")

# Optionally show a few of the missing ones
print("\nExample paperIds missing in df_author_cleaned:")
print(list(missing_in_author)[:10])

# Step 4: Remove those unmatched paperIds from both DataFrames to keep everything consistent
df_output_cleaned = df_output_cleaned[df_output_cleaned['paperId'].isin(paper_ids_author)]
df_author_cleaned = df_author_cleaned[df_author_cleaned['paperId'].isin(paper_ids_author)]

# Step 5: Drop exact duplicate author rows — we consider duplicates to be identical (paperId, country, researcherId)
before_dedup = len(df_author_cleaned)
df_author_cleaned = df_author_cleaned.drop_duplicates(
    subset=['paperId', 'country', 'researcherId'],
    keep='first'  # Keep the first occurrence
)
after_dedup = len(df_author_cleaned)
print(f"Removed {before_dedup - after_dedup} duplicate rows based on (paperId, country, researcherId).")
print(f"Remaining rows: {after_dedup}")

# Step 6: Clean authorSeq column — convert to numeric and fill missing values with -1
df_author_cleaned['authorSeq'] = pd.to_numeric(df_author_cleaned['authorSeq'], errors='coerce')  # Convert to numeric
missing_count = df_author_cleaned['authorSeq'].isna().sum()
print(f"Missing authorSeq values before fill: {missing_count}")
df_author_cleaned['authorSeq'] = df_author_cleaned['authorSeq'].fillna(-1).astype(int)  # Fill with -1 and convert to int
print(f"authorSeq column cleaned. Now using -1 for missing values.")

# Debug
df_author_cleaned.loc[
    (df_author_cleaned['fullName'] == 'Han, Rui') &
    (df_author_cleaned['paperId'] == 'WOS:000372420000001'),
    'researcherId'
] = "customID"

df_author_cleaned.loc[
    (df_author_cleaned['fullName'] == 'Guo, Yanshu') &
    (df_author_cleaned['paperId'] == 'WOS:001089303900010'),
    'researcherId'
] = "customID2"

# Final confirmation: show how many unique paperIds are in both cleaned DataFrames
print(f"\nAfter removing mismatched paperIds:")
print(f"Unique paperIds in df_output_cleaned: {df_output_cleaned['paperId'].nunique()}")
print(f"Unique paperIds in df_author_cleaned: {df_author_cleaned['paperId'].nunique()}")

Filtered df_output: 28283 → 28076 (2007–2023)
Filtered df_author: 194724 → 193376 (matching paperIds from df_output_cleaned)

Unique paperIds in df_output_cleaned: 28076
Unique paperIds in df_author_cleaned: 28033
PaperIds in df_output_cleaned but missing in df_author_cleaned: 43

Example paperIds missing in df_author_cleaned:
['WOS:000259566800001', 'WOS:000214815000012', 'WOS:000304507700006', 'WOS:000251053800010', 'WOS:000258760400001', 'WOS:000437852100006', 'WOS:000414069200006', 'WOS:000284196500002', 'WOS:000264397200009', 'WOS:000482989700061']
Removed 51543 duplicate rows based on (paperId, country, researcherId).
Remaining rows: 141833
Missing authorSeq values before fill: 1850
authorSeq column cleaned. Now using -1 for missing values.

After removing mismatched paperIds:
Unique paperIds in df_output_cleaned: 28033
Unique paperIds in df_author_cleaned: 28033


In [7]:
# Start fresh: initialize empty DataFrame for first-author selections
df_author_first_combined = pd.DataFrame(columns=df_author_cleaned.columns.tolist() + ['firstAuthorSource'])

# =======================================
# Strategy 1: duplicated row is done in previous codeblock
# =======================================

# =======================================
# Strategy 2: Unique authorSeq == 1
# =======================================

# Step 2.1: Filter candidates
df_first_author_candidates = df_author_cleaned[df_author_cleaned['authorSeq'] == 1]

# Step 2.2: Keep only paperIds with exactly one authorSeq==1
seq1_counts = df_first_author_candidates['paperId'].value_counts()
unique_seq1_paperIds = seq1_counts[seq1_counts == 1].index

# Step 2.3: Select rows
df_seq1_first = df_first_author_candidates[df_first_author_candidates['paperId'].isin(unique_seq1_paperIds)].copy()
df_seq1_first['firstAuthorSource'] = 'seq1_unique'

# Step 2.4: Add to combined
df_author_first_combined = pd.concat([df_author_first_combined, df_seq1_first], ignore_index=True)

print(f"\n✔ Strategy 2: Added {df_seq1_first.shape[0]} rows from unique authorSeq==1")
print(f"Shape: {df_author_first_combined.shape}")

# =======================================
# Strategy 3: Solo author, authorSeq == -1
# =======================================

# Step 3.1: Identify uncovered papers
covered_paper_ids = set(df_author_first_combined['paperId'])
df_author_remaining = df_author_cleaned[~df_author_cleaned['paperId'].isin(covered_paper_ids)]

# Step 3.2: Find solo-author papers
author_counts = df_author_cleaned['paperId'].value_counts()
solo_papers = author_counts[author_counts == 1].index

# Step 3.3: Select those with authorSeq == -1
df_fallback_first = df_author_remaining[
    (df_author_remaining['paperId'].isin(solo_papers)) &
    (df_author_remaining['authorSeq'] == -1)
].copy()
df_fallback_first['firstAuthorSource'] = 'fallback_-1_single'

# Step 3.4: Add to combined
df_author_first_combined = pd.concat([df_author_first_combined, df_fallback_first], ignore_index=True)

print(f"\n✔ Strategy 3: Added {df_fallback_first.shape[0]} fallback rows (solo authors, authorSeq == -1)")
print(f"Shape: {df_author_first_combined.shape}")

# =======================================
# Strategy 4: Same researcherId, multiple rows with authorSeq == 1
# =======================================

# Step 4.1: Filter uncovered + authorSeq == 1
covered_paper_ids = set(df_author_first_combined['paperId'])
df_author_remaining = df_author_cleaned[~df_author_cleaned['paperId'].isin(covered_paper_ids)]
df_seq1_remaining = df_author_remaining[df_author_remaining['authorSeq'] == 1]

# Step 4.2: Find paperId + researcherId combos with >1 row
duplicate_seq1_ids = (
    df_seq1_remaining.groupby(['paperId', 'researcherId'])
    .size().reset_index(name='count')
    .query('count > 1')
)

# Step 4.3: Merge rows for these combos
merged_rows = []
for _, row in duplicate_seq1_ids.iterrows():
    paper_id = row['paperId']
    researcher_id = row['researcherId']
    subset = df_seq1_remaining[
        (df_seq1_remaining['paperId'] == paper_id) &
        (df_seq1_remaining['researcherId'] == researcher_id)
    ]
    merged = subset.iloc[0].copy()
    merged['country'] = '; '.join(sorted(set(subset['country'].dropna())))
    merged['orgNames'] = '; '.join(sorted(set(subset['orgNames'].dropna())))
    merged['orgEnhanced1'] = '; '.join(sorted(set(subset['orgEnhanced1'].dropna())))
    merged['orgEnhancedAll'] = '; '.join(sorted(set(subset['orgEnhancedAll'].dropna())))
    merged['firstAuthorSource'] = 'merged_seq1_same_id'
    merged_rows.append(merged)

df_merged_seq1 = pd.DataFrame(merged_rows)
df_author_first_combined = pd.concat([df_author_first_combined, df_merged_seq1], ignore_index=True)

print(f"\n✔ Strategy 4: Merged and added {df_merged_seq1.shape[0]} rows (same researcherId, multiple affiliations)")
print(f"Shape: {df_author_first_combined.shape}")

# =======================================
# Strategy 5: Multi-author, all -1, no researcherId
# =======================================

# Step 5.1: Identify remaining uncovered papers
covered_paper_ids = set(df_author_first_combined['paperId'])
uncovered_paper_ids = set(df_author_cleaned['paperId']) - covered_paper_ids
df_remaining = df_author_cleaned[df_author_cleaned['paperId'].isin(uncovered_paper_ids)]

# Step 5.2: Filter where authorSeq == -1 and no researcherId
df_no_id = df_remaining[
    (df_remaining['authorSeq'] == -1) &
    (df_remaining['researcherId'].isna())
]

# Step 5.3: Group by paperId and merge metadata
grouped_rows = []
for paper_id, group in df_no_id.groupby('paperId'):
    merged = group.iloc[0].copy()
    merged['country'] = '; '.join(sorted(set(group['country'].dropna())))
    merged['orgNames'] = '; '.join(sorted(set(group['orgNames'].dropna())))
    merged['orgEnhanced1'] = '; '.join(sorted(set(group['orgEnhanced1'].dropna())))
    merged['orgEnhancedAll'] = '; '.join(sorted(set(group['orgEnhancedAll'].dropna())))
    merged['firstAuthorSource'] = 'merged_-1_no_id'
    grouped_rows.append(merged)

df_merged_fallback = pd.DataFrame(grouped_rows)
df_author_first_combined = pd.concat([df_author_first_combined, df_merged_fallback], ignore_index=True)

print(f"\n✔ Strategy 5: Merged and added {df_merged_fallback.shape[0]} fallback rows (multi-author -1, no researcherId)")
print(f"Shape: {df_merged_fallback.shape}")

# =======================================
# Strategy 6: Only one author, but not seq==1 (e.g. seq=-1)
# =======================================

# Step 6.1: Identify remaining uncovered paperIds
covered_paper_ids = set(df_author_first_combined['paperId'])
uncovered_paper_ids = set(df_author_cleaned['paperId']) - covered_paper_ids
df_remaining = df_author_cleaned[df_author_cleaned['paperId'].isin(uncovered_paper_ids)]

# Step 6.2: Count number of authors per paper
remaining_author_counts = df_remaining['paperId'].value_counts()
unique_author_paperIds = remaining_author_counts[remaining_author_counts == 1].index

# Step 6.3: Select the single remaining author rows
df_unique_author_remaining = df_remaining[df_remaining['paperId'].isin(unique_author_paperIds)].copy()
df_unique_author_remaining['firstAuthorSource'] = 'only_one_author_non_seq1'

# Step 6.4: Add to combined
df_author_first_combined = pd.concat([df_author_first_combined, df_unique_author_remaining], ignore_index=True)

print(f"\n✔ Strategy 6: Added {df_unique_author_remaining.shape[0]} rows (only one author, not seq==1)")
print(f"Shape: {df_author_first_combined.shape}")


# =======================================
# Strategy 7: No authorSeq == 1 → pick min authorSeq row (combine if tied)
# =======================================

# Step 7.1: Remaining uncovered paperIds
covered_paper_ids = set(df_author_first_combined['paperId'])
all_paper_ids = set(df_author_cleaned['paperId'])
uncovered_paper_ids = all_paper_ids - covered_paper_ids

# Step 7.2: Subset with those papers
df_uncovered = df_author_cleaned[df_author_cleaned['paperId'].isin(uncovered_paper_ids)]

# Step 7.3: Get paperIds that have no authorSeq == 1
paper_ids_no_seq1 = (
    df_uncovered.groupby('paperId')
    .filter(lambda g: (g['authorSeq'] != 1).all())
    .paperId.unique()
)

df_no_seq1 = df_uncovered[df_uncovered['paperId'].isin(paper_ids_no_seq1)].copy()

# Step 7.4: Process each paperId
merged_rows = []
for paper_id, group in df_no_seq1.groupby('paperId'):
    min_seq = group['authorSeq'].min()
    subset = group[group['authorSeq'] == min_seq]

    if len(subset) == 1:
        row = subset.iloc[0].copy()
        row['firstAuthorSource'] = 'strategy7_min_seq_single'
        merged_rows.append(row)
    else:
        merged = subset.iloc[0].copy()
        merged['country'] = '; '.join(sorted(set(subset['country'].dropna())))
        merged['orgNames'] = '; '.join(sorted(set(subset['orgNames'].dropna())))
        merged['orgEnhanced1'] = '; '.join(sorted(set(subset['orgEnhanced1'].dropna())))
        merged['orgEnhancedAll'] = '; '.join(sorted(set(subset['orgEnhancedAll'].dropna())))
        merged['firstAuthorSource'] = 'strategy7_min_seq_merged'
        merged_rows.append(merged)

df_strategy7 = pd.DataFrame(merged_rows)

# Step 7.5: Append to combined
df_author_first_combined = pd.concat([df_author_first_combined, df_strategy7], ignore_index=True)

# Final Check
print(f"\n✔ Strategy 7: Added {df_strategy7.shape[0]} rows (min authorSeq with merge if duplicate)")
print(f"Final shape after Strategy 7: {df_author_first_combined.shape}")
print(f"Unique paperIds in original: {df_author_cleaned['paperId'].nunique()}")
print(f"Unique paperIds in final: {df_author_first_combined['paperId'].nunique()}")



✔ Strategy 2: Added 24896 rows from unique authorSeq==1
Shape: (24896, 10)

✔ Strategy 3: Added 890 fallback rows (solo authors, authorSeq == -1)
Shape: (25786, 10)

✔ Strategy 4: Merged and added 1827 rows (same researcherId, multiple affiliations)
Shape: (27613, 10)

✔ Strategy 5: Merged and added 387 fallback rows (multi-author -1, no researcherId)
Shape: (387, 10)

✔ Strategy 6: Added 13 rows (only one author, not seq==1)
Shape: (28013, 10)

✔ Strategy 7: Added 20 rows (min authorSeq with merge if duplicate)
Final shape after Strategy 7: (28033, 10)
Unique paperIds in original: 28033
Unique paperIds in final: 28033


In [8]:
print(df_author_first_combined.shape)
print(df_output_cleaned.shape)

# Merge based on paperId
df_combined_full = df_output_cleaned.merge(
    df_author_first_combined,
    on='paperId',
    how='left',  # Keeps all rows from df_output_cleaned
    suffixes=('', '_author')  # Prevents name collision
)

print(f"Merged shape: {df_combined_full.shape}")

csv_path = os.path.join(output_folder, "df_final.csv")
df_combined_full.to_csv(csv_path, index=False)
print("Saved df_combined_full in csv folder")


(28033, 10)
(28033, 13)
Merged shape: (28033, 22)
Saved df_combined_full in csv folder


In [192]:
import pandas as pd
import plotly.express as px

# Starting with df_combined_full having 'country' and 'year' columns

# Step 1: Clean and split multi-country entries into multiple rows
df_map = df_combined_full[['country', 'year']].dropna()

# Remove extra spaces and split countries by ';'
df_map['country'] = df_map['country'].str.strip()

# Split country column by ';' and explode to separate rows
df_map = df_map.assign(country=df_map['country'].str.split(';')).explode('country')

# Clean spaces again after splitting
df_map['country'] = df_map['country'].str.strip()

# Step 2: Aggregate counts per country and year
df_grouped = (
    df_map.groupby(['year', 'country'])
    .size()
    .reset_index(name='publication_count')
)

def fix_uk_name(country):
    if country in ['UK', 'United Kingdom', 'England', 'Scotland', 'Wales', 'Northern Ireland']:
        return 'United Kingdom'
    return country

df_grouped['country'] = df_grouped['country'].apply(fix_uk_name)


max_publications = df_grouped['publication_count'].max()

# Step 3: Plot with bigger bubble map
fig = px.scatter_geo(
    df_grouped,
    locations='country',
    locationmode='country names',
    size='publication_count',
    hover_name='country',
    animation_frame='year',
    projection='natural earth',
    title='Publications per Country Over Time (Bubble Map)',
    size_max=80,
    color='publication_count',
    color_continuous_scale='Viridis',
    range_color=[0, max_publications],  # Fix color scale across frames
    labels={'publication_count': 'Number of Publications'},
)

fig.update_layout(
    width=1200,
    height=800,
    geo=dict(
        showframe=False,
        showcoastlines=False,
        showcountries=True,
    ),
    margin={"r":0,"t":40,"l":0,"b":0}
)

# Save HTML
fig.write_html("publications_bubble_map.html")

# Show plot
fig.show()
